## **IMPORT LIBRARY**

In [1]:
import os, warnings, math
import pandas as pd
import numpy as np
import pickle

# Machine learning Model 
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,classification_report,roc_curve,auc,average_precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

# Deep learning Model
import tensorflow as tf

# Directory Path Saving
os.chdir('..')
dir = os.getcwd()
print(dir)

2023-03-09 19:59:46.441148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 19:59:47.458017: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/senume/anaconda3/envs/Person_identification_Env/lib/
2023-03-09 19:59:47.458104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/senume/anaconda3/envs/Person_identification_Env/lib/
2023-03-09

/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial


In [2]:
warnings.filterwarnings("ignore")

## **FUNCTION DEFINITION**

In [3]:
# SVM
def SVM(X_Dataset,Y_Dataset, X_validation, Y_validation, Coeff, Interval):
  # Normally, C = 1 and gamma = 'scale' are default values
  # C controls how wide the margin will be with respect to how many misclassification we are allowing
  # C is increasing --> reduce the size of the margin and fewer misclassification and vice versa

  param_grid = [
      {'C': [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 30, 50, 80, 100],
      'gamma': ['scale', 0.5, 0.125, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001],
      'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
  ]



  optimal_params = GridSearchCV(SVC(),
                              param_grid,
                              scoring='f1_weighted',
                              cv=StratifiedKFold(5))

  optimal_params.fit(X_Dataset, Y_Dataset)
  print(optimal_params.best_params_)

  #svm
  C = optimal_params.best_params_['C']
  gamma = optimal_params.best_params_['gamma']
  kernel = optimal_params.best_params_['kernel']

  svm = SVC(C=C, gamma=gamma, kernel=kernel)
  svm.fit(X_Dataset, Y_Dataset)

  # predict the response
  svm_y_pred = svm.predict(X_validation)

  # Classification Report for the Support Vector Machine Model
  print("Classification Report : Support Vector Machine")
  classRep = classification_report(Y_validation, svm_y_pred, digits=5)

  ## Saving the Report and model
  pickle.dump(svm, open(dir + '/Record/' + f'Model_SVM_{Coeff}_{Interval}', 'wb'))
  np.savez_compressed(dir + '/Record/' + f'Model_SVM_Report_{Coeff}_{Interval}', Report = classRep)
  

  print(classRep)

## **LOADING DATASET**

In [4]:
Coeff_N = 9

os.chdir(dir + '/Feature_Dataset')
L = os.listdir()

# For given Interval Type
IntervalNo = 0
os.chdir(L[IntervalNo])
print('Interval Type: ', L[IntervalNo])

# Selecting Coefficient_Count Folder 
Coeff_Fol = os.listdir()
print("Chebyshev Coefficent",Coeff_Fol)
os.chdir(Coeff_Fol[Coeff_N])

# Patient List
PatientList =  os.listdir()

# Label Specifying Each Patient
Label = {}

for n in range(0,len(PatientList)):
    Label[PatientList[n]] = n

# Spliting the Person into Family List and New Individual List
N = len(PatientList)
FamilyList_EndIndex = int(math.ceil((70/100)*N))
FamilyList = PatientList[:FamilyList_EndIndex]
NewIdividualList = PatientList[FamilyList_EndIndex:]


# Reading Dataset
print("Reading Dataset of Chebyshev Coefficient: ", Coeff_Fol[Coeff_N])

Dataset = pd.DataFrame()


for name in FamilyList:
    
    Temp = pd.read_csv(name)
    Temp['TARGET'] = Label[name]
    Dataset = Dataset.append(Temp, ignore_index= True)
    
    


os.chdir(dir)

Interval Type:  PP_Interval
Chebyshev Coefficent ['20', '30', '40', '70', '50', '100', '60', '80', '90', '10']
Reading Dataset of Chebyshev Coefficient:  10


## **DATA ANALYSIS**

## **ML TRAINING**

### **SVM**

In [5]:
X = Dataset.loc[:,'CC_1':f'CC_{str(Coeff_Fol[Coeff_N])}']
Y = Dataset.loc[:,'TARGET']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size= 0.40, random_state=7823, stratify= Y)

SVM(X_train,Y_train.to_numpy(), X_validation, Y_validation.to_numpy(), Coeff=Coeff_Fol[Coeff_N], Interval=L[IntervalNo])


{'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf'}
Classification Report : Support Vector Machine
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        53
           1    1.00000   1.00000   1.00000        47

    accuracy                        1.00000       100
   macro avg    1.00000   1.00000   1.00000       100
weighted avg    1.00000   1.00000   1.00000       100

